In [23]:
from sklearn.preprocessing import 

ImportError: cannot import name 'Imputer' from 'sklearn.preprocessing' (/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/__init__.py)

In [5]:
from snowflake.snowpark import Session

In [21]:
import sklearn
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')
import configparser

# Code to establish connection and read data from Snowflake

In [7]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [8]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [9]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [10]:
application_train_sf  = session.table("CRA_APPLICATION_TRAIN_DETAILS")
application_test_sf  = session.table("CRA_APPLICATION_TEST_DETAILS")
bureau_sf  = session.table("CRA_BUREAU_DETAILS")
bureau_balance_sf  = session.table("CRA_BUREAU_BALANCE_DETAILS")
credit_card_balance_sf  = session.table("CRA_CREDIT_CARD_BALANCE_DETAILS")
installments_payments_sf  = session.table("CRA_INSTALLMENTS_PAYMENTS_DETAILS")
previous_application_sf  = session.table("CRA_PREVIOUS_APPLICATION_DETAILS")
POS_CASH_balance_sf  = session.table("CRA_POS_CASH_BALANCE_DETAILS")

# Convert Snowflake data into Pandas dataframes

In [11]:
application_train = application_train_sf.to_pandas()

# Glimpse of Rows and Columns of each dataset

In [12]:
print("application_train -  rows:",application_train.shape[0]," columns:", application_train.shape[1])

application_train -  rows: 307511  columns: 124


# Sample Data of each dataset

In [13]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREATED_BY,CREATED_AT
0,307660,0,Cash loans,M,Y,Y,0,180000.0,1288350.0,41692.5,...,0,0,0.0,0.0,0.0,0.0,1.0,2.0,manishh.singh@fosfor.com,2024-05-03 06:30:22
1,307662,0,Cash loans,F,N,Y,1,90000.0,93829.5,9981.0,...,0,0,0.0,0.0,0.0,0.0,0.0,4.0,manishh.singh@fosfor.com,2024-05-03 06:30:22
2,307663,0,Cash loans,F,N,Y,0,360000.0,2517300.0,66402.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,manishh.singh@fosfor.com,2024-05-03 06:30:22
3,307664,0,Revolving loans,F,N,N,0,90000.0,270000.0,13500.0,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,manishh.singh@fosfor.com,2024-05-03 06:30:22
4,307665,0,Revolving loans,M,Y,Y,0,225000.0,540000.0,27000.0,...,0,0,0.0,0.0,0.0,0.0,2.0,7.0,manishh.singh@fosfor.com,2024-05-03 06:30:22


In [16]:
application_train.drop(['CREATED_BY','CREATED_AT'], inplace=True, axis=1)

In [19]:
print("application_train -  rows:",application_train.shape[0]," columns:", application_train.shape[1])

application_train -  rows: 307511  columns: 122


# Stratified Sampling (ratio=0.1)

In [17]:
application_sample1 = application_train.loc[application_train.TARGET==1].sample(frac=0.1, replace=False)
print('label 1 sample size:', str(application_sample1.shape[0]))
application_sample0 = application_train.loc[application_train.TARGET==0].sample(frac=0.1, replace=False)
print('label 0 sample size:', str(application_sample0.shape[0]))
application = pd.concat([application_sample1, application_sample0], axis=0).sort_values('SK_ID_CURR')

label 1 sample size: 2482
label 0 sample size: 28269


## Impute missing value

In [18]:
categorical_list = []
numerical_list = []
for i in application.columns.tolist():
    if application[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print('Number of categorical features:', str(len(categorical_list)))
print('Number of numerical features:', str(len(numerical_list)))

Number of categorical features: 16
Number of numerical features: 106


In [22]:
from sklearn.preprocessing import Imputer
application[numerical_list] = Imputer(strategy='median').fit_transform(application[numerical_list])

ImportError: cannot import name 'Imputer' from 'sklearn.preprocessing' (/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/__init__.py)